<a href="https://colab.research.google.com/github/OlyKoek/MyStudy-AI-app/blob/colab/create_multimodal_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルチモーダルなエンベディングモデルの作成

#### 日本語表示対応

In [ ]:
# 日本語フォントをダウンロードする。
!apt-get -y install fonts-ipafont-gothic
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'IPAGothic'

import shutil
import os
# フォントキャッシュを削除
font_cache_path = os.path.expanduser("~/.cache/matplotlib")
if os.path.exists(font_cache_path):
    shutil.rmtree(font_cache_path)


### Pytorchのパッケージと必要なライブラリをインストール
- torch   
- transformers from hugging face
- umap-learn : 可視化で使ってみたい。t-SNEよりも良いらしい
- matplotlib : ベクトル空間の可視化に使用

In [ ]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers umap-learn matplotlib pandas scikit-learn --quiet

# 概要
やりたいことは、マルチモーダルな入力をベクトル空間に投影すること。
今回はできるだけ簡単に構築したいので、下記の構成。

- テキスト：TinyBERT系   
(最初all-MiniLM-L6-v2を使ったけど、日本語使えないことに気が付いたので一旦sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2に変更。東北大学が作った日本語専用のbert-base-japanese-v2もよさげだけど。一旦これで行く。)
- 画像：CLIP
- 音：MCFF

## Step1 テキストのEmbedding

ここでは、大規模コーパスで学習済のTinyBERTを**特徴量抽出器**として扱い、
その出力をLiner層(Projection Head)を使って、マルチモーダル用の次元圧縮と
好きな意味空間になるように調整を行う。ざっくり言うと、プロジェクションヘッドを使った転移学習。

step1: TinyBERTにテキストを入力してエンコード   
step2: エンコード結果をLiner層で次元圧縮しつつ、任意の意味空間になるような学習をさせる    
Step3: 好きなテキストをTinyBERT+Linerに入れると、Step2の学習に従ったベクトル意味空間にプロットされる。

### ■ tinyBERTをダウンロード

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


# read TinyBERT model
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)



(動作確認用)

In [ ]:
# @title
# test sample text
texts = ["今日めっちゃ良かった楽しい","マジつまんね。つらい","今日は普通かな特に何もないし"]

# tokenize
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


# test ==========================
# view to tokenizerd text
print("-"*50)
text = texts[0]
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Tokenizeの出力確認\n\n 入力文章：{text} \nトークン文字：{tokens} \nトークンID: {ids}")
print("-"*50)
# ==============================


# input tokenized data to TinyBERT model
with torch.no_grad():
    outputs = model(**inputs)

# convert token to vector
embeddings = outputs.last_hidden_state.mean(dim=1)
print("\n文数 =",embeddings.shape[0]," ベクトル次元 =", embeddings.shape[1])

### ■ Linearヘッドを追加して256次元に射影

 射影：今回使ったTinyBERTのディフォルトの埋め込みベクトル次元空間を、別のベクトル次元目的空間に変換するための、線形変換を適応

 今回はテキスト以外も同じ特徴空間にプロットするので、共有可能なベクトル次元空間にする必要がある。


In [ ]:
import torch.nn as nn

projector = nn.Linear(embeddings.shape[1], 256)

#射影
projected = projector(embeddings)
print(projected.shape)

#### ■ 対象学習　ベクトル同士の意味付け

似た意味の単語を近く人、反対の意味の単語を遠くに、という形でベクトル空間上のプロットを最適化して、自然にクラスタ化されるようにする。


In [ ]:
import pandas as pd

data = {
    "anchor": ["嬉しい", "悲しい", "疲れた", "ワクワクする", "寂しい", "楽しい"],
    "positive": ["楽しい", "つらい", "だるい", "楽しみ", "悲しい", "嬉しい"],
    "negative": ["怒っている", "嬉しい", "ワクワクする", "眠い", "明るい", "悲しい"]
}
df = pd.DataFrame(data)
df

In [ ]:
from torch.nn import functional as F

def encode_texts(text_list):
  inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# 各文をエンコード
anchor_emb = encode_texts(df["anchor"].tolist())
pos_emb = encode_texts(df["positive"].tolist())
neg_emb = encode_texts(df["negative"].tolist())

# 正例：anchorとposを近づける
pos_sim = F.cosine_similarity(anchor_emb, pos_emb)
# 負例：anchorとnegを離す
neg_sim = F.cosine_similarity(anchor_emb, neg_emb)

# InfoNCE風のloss（シンプル版）
loss = -torch.log(torch.exp(pos_sim) / (torch.exp(pos_sim) + torch.exp(neg_sim)))
loss = loss.mean()
print(f"Contrastive Loss: {loss.item():.4f}")

  ■ 調整したベクトル間の関係でLinerを学習

In [ ]:
optimizer = torch.optim.AdamW(projector.parameters(), lr=1e-4)

for epoch in range(10):
    optimizer.zero_grad()
    anchor_emb = encode_texts(df["anchor"].tolist())
    pos_emb = encode_texts(df["positive"].tolist())
    neg_emb = encode_texts(df["negative"].tolist())

    pos_sim = F.cosine_similarity(projector(anchor_emb), projector(pos_emb))
    neg_sim = F.cosine_similarity(projector(anchor_emb), projector(neg_emb))
    loss = -torch.log(torch.exp(pos_sim) / (torch.exp(pos_sim) + torch.exp(neg_sim))).mean()

    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")


In [ ]:
# @title
all_texts = df["anchor"].tolist() + df["positive"].tolist() + df["negative"].tolist()
embeddings = encode_texts(all_texts)
projected = projector(embeddings).detach().numpy()

import umap, matplotlib.pyplot as plt
reducer = umap.UMAP(metric="cosine", n_neighbors=5, min_dist=0.3)
umap_emb = reducer.fit_transform(projected)

plt.figure(figsize=(6,6))
plt.scatter(umap_emb[:,0], umap_emb[:,1], c="orange", alpha=0.7)
for i, t in enumerate(all_texts):
    plt.text(umap_emb[i,0], umap_emb[i,1], t, fontsize=9)
plt.title("Emotion Space after Contrastive Learning")
plt.show()


## Step2 画像のEmbedding

ここでは、大規模コーパスで学習済のTinyBERTを**特徴量抽出器**として扱い、
その出力をLiner層(Projection Head)を使って、マルチモーダル用の次元圧縮と
好きな意味空間になるように調整を行う。ざっくり言うと、プロジェクションヘッドを使った転移学習。

step1: TinyBERTにテキストを入力してエンコード   
step2: エンコード結果をLiner層で次元圧縮しつつ、任意の意味空間になるような学習をさせる    
Step3: 好きなテキストをTinyBERT+Linerに入れると、Step2の学習に従ったベクトル意味空間にプロットされる。

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import requests
from io import BytesIO

# MobileNetV3 Small
image_model = models.mobilenet_v3_small(pretrained=True)
image_model.eval()  # 使用するエンコーダは凍結

# 最終層直前の512次元ベクトルを得るためのhook
# MobileNetV3の特徴抽出部だけを使う
feature_extractor = image_model.features

# 画像前処理
image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# 画像のエンコード用関数を作成
def encode_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    tensor = image_transform(img).unsqueeze(0)  # [1,3,224,224]

    with torch.no_grad():
        feat = feature_extractor(tensor)  # [1, 576, 7,7]
        feat = torch.mean(feat, dim=[2,3])  # 平均プーリング → [1, 576]

    return feat


（動作確認用）

In [ ]:
# @title
# テスト画像（犬）
test_url = "https://images.dog.ceo/breeds/chow/n02112137_16777.jpg"
img_emb = encode_image(test_url)
print(img_emb.shape)

# Step3 各種エンコーダーを整理して、それぞれの入力の関連性を使った学習の準備。
同じ特徴空間を指していることを確認。


In [ ]:
import torch.nn as nn

# MobileNetV3-Small の最終特徴量は 576 次元
image_projector = nn.Linear(576, 256)
# TinyBERT 用 projector（すでにあるはず）
text_projector = projector  # 前フェーズで作った 384 → 256 Linear


# 学習用のエンコーダー用関数を作成
def encode_text_for_train(text):
    emb = encode_texts([text])           # TinyBERT（固定）
    proj = text_projector(emb)           # projector（学習対象）
    proj = proj / (proj.norm(dim=-1, keepdim=True) + 1e-8)
    return proj

def encode_image_for_train(url):
    feat = encode_image(url)             # MobileNet（固定）
    proj = image_projector(feat)         # projector（学習対象）
    proj = proj / (proj.norm(dim=-1, keepdim=True) + 1e-8)
    return proj

# 推論用のエンコーダー用関数を作成
def encode_text_256(text):
    with torch.no_grad():
        emb = encode_texts([text])      # TinyBERT → [1,384]
        proj = text_projector(emb)      # projector → [1,256]
        proj = proj / (proj.norm(dim=-1, keepdim=True) + 1e-8)
    return proj

def encode_image_256(url):
    with torch.no_grad():
        feat = encode_image(url)        # MobileNet → [1,576]
        proj = image_projector(feat)    # projector → [1,256]
        proj = proj / (proj.norm(dim=-1, keepdim=True) + 1e-8)
    return proj


■ データセット

In [ ]:
image_text_pairs = [
    {
        "url": "https://images.dog.ceo/breeds/chow/n02112137_16777.jpg",
        "text": "白いフワフワの犬"
    },
    {
        "url": "https://images.dog.ceo/breeds/pitbull/20190710_143021.jpg",
        "text": "黒と白の犬が緑の草原の中にいる"
    },
    {
        "url": "https://images.dog.ceo/breeds/poodle-toy/n02113624_9550.jpg",
        "text": "ふわふわの子犬の写真"
    },
    {
        "url": "https://images.dog.ceo/breeds/eskimo/n02109961_8185.jpg",
        "text": "白い犬が石畳の上に座っている"
    }
]

img_vec = encode_image_256(image_text_pairs[0]["url"])
txt_vec = encode_text_256(image_text_pairs[0]["text"])
print(img_vec.shape, txt_vec.shape)

■　対照学習

In [ ]:
# clipを参考に事前に作成した、projection head層だけを学習する。
optimizer = torch.optim.AdamW(
    list(image_projector.parameters()) +
    list(text_projector.parameters()),
    lr=1e-4
)


# Enbeddingされた画像ベクトルとテキストベクトルの内積を全部取得
def contrastive_loss(img_vecs, txt_vecs):
    # cosine similarity matrix
    sim_matrix = torch.matmul(
        img_vecs,
        txt_vecs.T
    )  # [batch, batch]

    # 温度パラメータ（CLIP論文と同じ）
    temperature = 0.07
    sim_matrix = sim_matrix / temperature

    # 正解 = 対角線ペア
    labels = torch.arange(len(sim_matrix)).long()

    # 学習データ通り画像とテキストのペアで学習されるようにする。
    loss_i = torch.nn.CrossEntropyLoss()(sim_matrix, labels)
    loss_t = torch.nn.CrossEntropyLoss()(sim_matrix.T, labels)
    return (loss_i + loss_t) / 2


# 学習ループ
for epoch in range(20):
    optimizer.zero_grad()

    img_vecs = []
    txt_vecs = []

    for pair in image_text_pairs:
        img_vecs.append(encode_text_for_train(pair["text"]))
        txt_vecs.append(encode_image_for_train(pair["url"]))

    img_vecs = torch.cat(img_vecs, dim=0)
    txt_vecs = torch.cat(txt_vecs, dim=0)

    loss = contrastive_loss(img_vecs, txt_vecs)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")


(動作確認用)

In [ ]:
# @title
query = "ふわふわの犬"
q_vec = encode_text_256(query)

sims = torch.nn.functional.cosine_similarity(
    q_vec, img_vecs
)
print(sims)


In [ ]:
# @title
import numpy as np
import umap
import matplotlib.pyplot as plt

all_vecs = []
labels = []

# text
for pair in image_text_pairs:
    all_vecs.append(encode_text_256(pair["text"]).detach().numpy())
    labels.append("txt:" + pair["text"])

# image
for pair in image_text_pairs:
    all_vecs.append(encode_image_256(pair["url"]).detach().numpy())
    labels.append("img")

all_vecs = np.vstack(all_vecs)

reducer = umap.UMAP(metric="cosine")
umap_vecs = reducer.fit_transform(all_vecs)

plt.figure(figsize=(8,6))
for i, lab in enumerate(labels):
    plt.scatter(umap_vecs[i,0], umap_vecs[i,1])
    plt.text(umap_vecs[i,0], umap_vecs[i,1], lab[:10], fontsize=8)

plt.title("Mini-CLIP: Image × Text Shared Space")
plt.show()


## ベクトルDBを作成

In [ ]:
vector_db = []  # ここにすべての埋め込みを保存

def build_vector_db(pairs):
    for pair in pairs:
        # テキスト埋め込み
        txt_vec = encode_text_256(pair["text"]).cpu().detach().numpy()
        vector_db.append({
            "type": "text",
            "text": pair["text"],
            "url": pair["url"],   # 画像と紐付いてる場合
            "vec": txt_vec
        })

        # 画像埋め込み
        img_vec = encode_image_256(pair["url"]).cpu().detach().numpy()
        vector_db.append({
            "type": "image",
            "text": pair["text"],
            "url": pair["url"],
            "vec": img_vec
        })

build_vector_db(image_text_pairs)
len(vector_db)

## Vector DBを使用するための関数を作成

In [ ]:
import numpy as np
from PIL import Image
import requests
from IPython.display import display

def cosine_sim(a, b):
    # Ensure inputs are 1D arrays for dot product and norm calculation
    a_flat = a.flatten()
    b_flat = b.flatten()
    return np.dot(a_flat, b_flat) / (np.linalg.norm(a_flat) * np.linalg.norm(b_flat) + 1e-8)

def search_db(query_vec, top_k=5, mode=None):
    results = []
    for item in vector_db:

        if mode is not None:
            if item["type"] != mode:  # "image" or "text"
                continue

        sim = cosine_sim(query_vec, item["vec"])
        results.append((sim, item))

    results.sort(reverse=True, key=lambda x: x[0])
    return results[:top_k]

def pretty_print_results(results):
    for sim, item in results:
        img = Image.open(requests.get(item["url"], stream=True).raw)
        new_size = (150, 100)
        img = img.resize(new_size, Image.Resampling.LANCZOS)
        display(img)
        print(f"sim={sim:.3f} | type={item['type']} | text={item['text']} | url={item['url']}")

■　テスト

In [ ]:
query = "白い犬"
q_vec = encode_text_256(query).cpu().detach().numpy()

results = search_db(q_vec, top_k=5)
pretty_print_results(results)


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import umap

embs = np.array([item["vec"].flatten() for item in vector_db])
labels = [item["type"] for item in vector_db]
texts = [item["text"] for item in vector_db]
urls = [item["url"] for item in vector_db]

reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, metric="cosine")
coords = reducer.fit_transform(embs)  # shape (N, 2)

df = pd.DataFrame({
    "x": coords[:,0],
    "y": coords[:,1],
    "type": labels,
    "text": texts,
    "url": urls,
})

fig = px.scatter(
    df,
    x="x", y="y",
    color="type",          # text / image を色分け
    hover_data=["text", "url", "type"],
    title="UMAP Visualization (Interactive)",
    width=800, height=600
)

fig.show()
